1、torch.autograd，自动微分，支持对任意计算图的自动计算

后向传播，基于loss函数对每个参数的梯度进行更新

In [1]:
#定义一个简单的一层神经网络
import torch

x=torch.ones(5) # input tensor
y=torch.zeros(3) # expected output 期望输出
w=torch.randn(5,3,requires_grad=True)
b=torch.randn(3,requires_grad=True)
z=torch.matmul(x,w)+b 
loss=torch.nn.functional.binary_cross_entropy_with_logits(z,y)

不断调节参数，使得预测值和真实值越接近越好

调节参数一般用梯度后向传播算法

In [2]:
print(f"Gradient function for z={z.grad_fn}")
print(f"Gradient function for loss={loss.grad_fn}") #二分类交叉熵

Gradient function for z=<AddBackward0 object at 0x00000272E2F297F0>
Gradient function for loss=<BinaryCrossEntropyWithLogitsBackward0 object at 0x00000272E2F290B8>


2、算出每个参数的梯度

In [3]:
# 根节点loss对所有的父节点和祖辈节点进行梯度的运算
# loss必须是标量才可以这样调用
loss.backward() #加入retain_graph=True可以保留图
print(w.grad)   #loss关于w的梯度
print(b.grad)   #loss关于b的梯度

tensor([[0.1151, 0.3126, 0.0219],
        [0.1151, 0.3126, 0.0219],
        [0.1151, 0.3126, 0.0219],
        [0.1151, 0.3126, 0.0219],
        [0.1151, 0.3126, 0.0219]])
tensor([0.1151, 0.3126, 0.0219])


3、微调、推理或做测试，只需要做前向运算，而不需要做梯度的运算时，

In [4]:
#可用torch.no_grad()上下文控制
z=torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
    z=torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [5]:
#可用detach分离
z=torch.matmul(x,w)+b
z_det=z.detach()
print(z_det.requires_grad)

False


4、PyTorch默认情况下梯度可累积，调用zero_grad清零

5、雅可比，对长度大于1的张量求梯度

In [6]:
inp=torch.eye(5,requires_grad=True)
out=(inp+1).pow(2)
out.backward(torch.ones_like(inp),retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp),retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(inp),retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
